# 🧾 **AI-Assisted Tax Audit & Research Copilot**

This notebook demonstrates an **AI-assisted tax research and audit support system** designed to help tax auditors and regulatory authorities work faster, more consistently, and with higher confidence.

The system allows auditors to:
- Ask tax-related questions or describe audit scenarios in natural language
- Automatically retrieve relevant tax laws, regulations, and official guidance
- Receive a **structured, audit-ready summary** with clear citations to source documents

## Business Problem

Tax auditors and regulatory authorities must work with a **large and continuously evolving body of tax laws, executive regulations, and official guidance**.

Today, this information is:
- Spread across multiple documents and formats
- Frequently updated through amendments and clarifications
- Often searched and summarized manually

As a result:
- Auditors spend significant time locating relevant provisions
- There is a higher risk of missing applicable rules or using outdated guidance
- Scaling audit quality across teams becomes difficult

This creates a clear need for an **automated assistance system** that supports auditors with fast, reliable, and transparent access to tax information.


In [1]:
!pip -q install --upgrade \
  openai==1.66.3 \
  langchain>=1.0.0 \
  langchain-core>=1.0.0 \
  langchain-openai>=0.3.0 \
  langchain-community>=0.3.0 \
  faiss-cpu \
  sentence-transformers \
  pypdf \
  python-docx \
  gradio \
  pandas

## Secure Model Access

The system uses a secured, OpenAI-compatible gateway.  
Credentials are loaded from Colab Secrets and are never hard-coded.


In [2]:
from google.colab import userdata

API_KEY = userdata.get("OPEN_AI_API")
assert API_KEY, "Missing Colab Secret: OPEN_AI_API"

BASE_URL = "https://aibe.mygreatlearning.com/openai/v1"
print("✅ Key loaded and gateway set:", BASE_URL)


✅ Key loaded and gateway set: https://aibe.mygreatlearning.com/openai/v1


In [3]:
import os, io, re, json, traceback
from typing import List, Dict, Any, Tuple

from pypdf import PdfReader
import docx

## Approach Overview

The prototype follows a **retrieval-first approach** to audit support.

Instead of generating answers from general knowledge, the system:
1. Ingests official tax documents provided by the auditor
2. Breaks them into searchable sections
3. Retrieves only the most relevant excerpts for a given question
4. Generates a structured audit summary based strictly on those excerpts

Several design decisions were made deliberately:
- The system is **source-grounded** and refuses to answer when evidence is missing
- All retrieved sources are visible and reviewable
- Outputs follow a consistent structure aligned with audit documentation

This approach prioritizes **traceability, transparency, and control**.

In [4]:
def read_file_bytes(filename: str, file_bytes: bytes) -> str:
    name = filename.lower()

    if name.endswith(".pdf"):
        reader = PdfReader(io.BytesIO(file_bytes))
        pages = []
        for i, page in enumerate(reader.pages):
            txt = page.extract_text() or ""
            pages.append(f"[PAGE {i+1}] {txt}")
        text = "\n".join(pages)

    elif name.endswith(".docx"):
        d = docx.Document(io.BytesIO(file_bytes))
        text = "\n".join(p.text for p in d.paragraphs)

    else:
        text = file_bytes.decode("utf-8", errors="ignore")

    text = re.sub(r"\s+", " ", text).strip()
    return text


## Demo Documents

For demonstration purposes, the notebook generates a small set of **synthetic tax documents** inside the Colab environment.

These documents simulate laws, regulations, and official guidance and are used solely to demonstrate the workflow end-to-end.

From the system’s perspective, demo documents are treated exactly the same as real documents uploaded by an auditor.


In [5]:
!pip -q install reportlab

In [6]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from pathlib import Path
import textwrap, datetime, zipfile

out_dir = Path("tax_demo_docs")
out_dir.mkdir(exist_ok=True)

def make_pdf(path, title, sections):
    c = canvas.Canvas(str(path), pagesize=letter)
    w, h = letter
    x, y = 0.75*inch, h - 0.9*inch

    c.setFont("Helvetica-Bold", 16)
    c.drawString(x, y, title)
    y -= 0.4*inch

    c.setFont("Helvetica", 9)
    c.drawString(x, y, f"SYNTHETIC DEMO DOCUMENT — {datetime.date.today()} (NOT REAL LAW)")
    y -= 0.3*inch

    for header, body in sections:
        if y < 1.2*inch:
            c.showPage()
            y = h - 0.9*inch

        c.setFont("Helvetica-Bold", 12)
        c.drawString(x, y, header)
        y -= 0.25*inch

        c.setFont("Helvetica", 11)
        for line in textwrap.wrap(body, 95):
            if y < 1.0*inch:
                c.showPage()
                y = h - 0.9*inch
            c.drawString(x, y, line)
            y -= 0.18*inch
        y -= 0.15*inch

    c.save()

# --- Synthetic VAT Law ---
make_pdf(
    out_dir / "VAT_Law_Demo.pdf",
    "Synthetic VAT Law (Demo)",
    [
        ("Article 12 – Registration Threshold",
         "Mandatory VAT registration applies if taxable supplies exceed AED 375,000 "
         "in the preceding 12 months. Voluntary registration applies from AED 187,500."),
        ("Article 22 – Filing Deadline",
         "VAT returns must be submitted no later than the 28th day following the end "
         "of the tax period."),
        ("Article 59 – Late Filing Penalty",
         "AED 1,000 for the first late return. AED 2,000 for repeated late returns "
         "within 24 months."),
        ("Article 60 – Late Payment Penalty",
         "2% immediately after the due date, 4% after 7 days, plus 1% daily thereafter.")
    ]
)

# --- Synthetic Regulation ---
make_pdf(
    out_dir / "VAT_Regulation_Demo.pdf",
    "Synthetic VAT Executive Regulation (Demo)",
    [
        ("Regulation 7 – Small Business Supplies",
         "Persons below the mandatory registration threshold are not required to "
         "charge VAT unless voluntarily registered."),
        ("Regulation 52 – Penalty Mitigation",
         "Penalties may be reduced if a justified excuse is accepted by the authority.")
    ]
)

# --- Synthetic Guidance ---
make_pdf(
    out_dir / "VAT_Guidance_Demo.pdf",
    "Synthetic Tax Authority Guidance (Demo)",
    [
        ("GN-07 – Late Filing Review",
         "Auditors should verify submission timestamps and assigned tax periods."),
        ("GN-07 – Audit Checklist",
         "Check filing history, payment confirmations, turnover evidence, and "
         "mitigation requests.")
    ]
)

# ZIP
zip_path = out_dir / "tax_demo_docs.zip"
with zipfile.ZipFile(zip_path, "w") as z:
    for f in out_dir.glob("*.pdf"):
        z.write(f, f.name)

print("✅ Files created in Colab:")
for f in out_dir.iterdir():
    print(" -", f)


✅ Files created in Colab:
 - tax_demo_docs/VAT_Law_Demo.pdf
 - tax_demo_docs/tax_demo_docs.zip
 - tax_demo_docs/VAT_Regulation_Demo.pdf
 - tax_demo_docs/VAT_Guidance_Demo.pdf


## Audit Workflow

The workflow mirrors how auditors typically prepare and analyze source material:

1. Relevant documents are uploaded (or loaded from demo data)
2. Text is extracted and indexed for retrieval
3. The auditor enters a question or audit scenario
4. The system retrieves relevant provisions and generates a structured memo
5. Source evidence can be inspected directly within the interface

This keeps the auditor in control while reducing manual effort.


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=150,
    separators=["\n\n", "\n", "Article ", "Section ", ". ", " "]
)

VECTORSTORE = None

def build_index(files: List[Dict[str, Any]]) -> str:
    global VECTORSTORE

    docs = []
    for f in files:
        text = read_file_bytes(f["name"], f["bytes"])
        if len(text) < 50:
            # warn but keep going (some docs may have little text)
            print(f"⚠️ Low text extracted from {f['name']} (len={len(text)})")
        docs.append(Document(page_content=text, metadata={"source": f["name"]}))

    chunks = splitter.split_documents(docs)
    VECTORSTORE = FAISS.from_documents(chunks, embeddings)

    return f"✅ Indexed {len(files)} file(s) into {len(chunks)} chunks."


/tmp/ipython-input-1194074569.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="gpt-4o-mini",     # If your gateway requires another model name, change here
    temperature=0.1,
    api_key=API_KEY,
    base_url=BASE_URL
)

PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an AI-assisted tax research and audit support assistant.

Rules (STRICT):
- Use ONLY the provided SOURCES (snippets). Do not use outside knowledge.
- Every factual statement MUST have a citation like [source | chunk_id].
- If sources are insufficient, say "INSUFFICIENT EVIDENCE" and list what is missing.
- Prefer the most authoritative sources if there is conflict.

Return ONLY valid JSON with keys:
summary (string),
key_provisions (array of objects: {{ "point": string, "citation": string }}),
obligations (array of objects: {{ "item": string, "citation": string }}),
exemptions (array of objects: {{ "item": string, "citation": string }}),
penalties (array of objects: {{ "item": string, "citation": string }}),
audit_checklist (array of strings),
assumptions (array of strings),
sources_used (array of strings)."""),

    ("human", """AUDIT DATE (as-of): {as_of_date}

QUERY / SCENARIO:
{query}

SOURCES:
{sources}

Return ONLY JSON. No markdown. No commentary.""")
])


In [9]:
def _safe_json_parse(text: str) -> Tuple[Dict[str, Any], str]:
    raw = (text or "").strip()

    try:
        return json.loads(raw), raw
    except Exception:
        pass

    # Try to extract JSON object if wrapped with extra text
    start = raw.find("{")
    end = raw.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = raw[start:end+1]
        try:
            return json.loads(candidate), raw
        except Exception:
            pass

    return {"error": "Model returned non-JSON or invalid JSON.", "raw_output": raw[:6000]}, raw


def retrieve_with_scores(query: str, k: int = 6):
    global VECTORSTORE
    if VECTORSTORE is None:
        return []
    results = VECTORSTORE.similarity_search_with_score(query, k=k)
    out = []
    for i, (doc, score) in enumerate(results, start=1):
        doc.metadata["chunk_id"] = f"r{i}"
        out.append((doc, float(score)))
    return out


def answer_query(query: str, as_of_date: str = "Today", k: int = 6) -> Dict[str, Any]:
    global VECTORSTORE
    if VECTORSTORE is None:
        return {"error": "No index found. Upload documents and click Build Index first."}

    retrieved = retrieve_with_scores(query, k=k)
    if not retrieved:
        return {"error": "No retrieved sources. PDFs may have no extractable text."}

    sources_blocks = []
    for i, (d, score) in enumerate(retrieved, start=1):
        src = d.metadata.get("source", "unknown")
        cid = d.metadata.get("chunk_id", f"r{i}")
        snippet = d.page_content[:1200]
        sources_blocks.append(f"SNIPPET {i} [{src} | {cid} | score={score:.4f}]: {snippet}")

    sources_text = "\n\n".join(sources_blocks)

    try:
        msg = PROMPT.format_messages(query=query, as_of_date=as_of_date, sources=sources_text)
        out_text = llm.invoke(msg).content
    except Exception as e:
        return {"error": f"LLM call failed: {type(e).__name__}: {e}"}

    parsed, raw = _safe_json_parse(out_text)

    # Attach sources for UI transparency
    parsed["_retrieved_sources"] = [
        {
            "rank": i,
            "source": d.metadata.get("source", "unknown"),
            "chunk_id": d.metadata.get("chunk_id", f"r{i}"),
            "score": float(score),
            "preview": d.page_content[:350].replace("\n", " ")
        }
        for i, (d, score) in enumerate(retrieved, start=1)
    ]

    return parsed


In [10]:
def json_to_memo_md(result: dict) -> str:
    if not isinstance(result, dict):
        return "## ❌ Error\n\nUnexpected result type."

    if "error" in result:
        tb = result.get("traceback", "")
        raw = result.get("raw_output", "")
        return (
            "## ❌ Error\n\n"
            f"**{result['error']}**\n\n"
            + (f"### Traceback\n```text\n{tb}\n```\n" if tb else "")
            + (f"### Raw Output\n```text\n{raw}\n```\n" if raw else "")
        )

    def bullets(items, key="item"):
        if not items:
            return "_None found in provided sources._"
        out = []
        for x in items:
            if isinstance(x, dict):
                text = x.get(key) or x.get("point") or ""
                cit = x.get("citation", "")
                out.append(f"- {text} **{cit}**" if cit else f"- {text}")
            else:
                out.append(f"- {x}")
        return "\n".join(out)

    md = []
    md.append("# 🧾 Audit Research Memo")
    md.append("")
    md.append("## Summary")
    md.append(result.get("summary", "_No summary returned._"))
    md.append("")
    md.append("## Key Provisions")
    md.append(bullets(result.get("key_provisions", []), key="point"))
    md.append("")
    md.append("## Obligations")
    md.append(bullets(result.get("obligations", []), key="item"))
    md.append("")
    md.append("## Exemptions / Thresholds")
    md.append(bullets(result.get("exemptions", []), key="item"))
    md.append("")
    md.append("## Penalties")
    md.append(bullets(result.get("penalties", []), key="item"))
    md.append("")
    md.append("## Audit Checklist")
    checklist = result.get("audit_checklist", [])
    md.append("\n".join([f"- {x}" for x in checklist]) if checklist else "_None._")
    md.append("")
    md.append("## Assumptions")
    assumptions = result.get("assumptions", [])
    md.append("\n".join([f"- {x}" for x in assumptions]) if assumptions else "_None._")
    md.append("")
    md.append("## Sources Used")
    srcs = result.get("sources_used", [])
    md.append("\n".join([f"- {x}" for x in srcs]) if srcs else "_See Retrieved Sources panel._")

    return "\n".join(md)


In [11]:
from pathlib import Path

out_dir = Path("tax_demo_docs")  # adjust if your demo docs folder is different

initial_uploaded_files_store = []
if out_dir.exists():
    for f_path in out_dir.glob("*.pdf"):
        with open(f_path, "rb") as f:
            initial_uploaded_files_store.append({"name": f_path.name, "bytes": f.read()})

print(f"✅ Demo files found: {len(initial_uploaded_files_store)}")

✅ Demo files found: 3


In [12]:
uploaded_files_store = []  # populated by ui_upload or fallback to demo files

def ui_upload(files) -> str:
    global uploaded_files_store
    try:
        uploaded_files_store = []

        if not files:
            return "No new files selected. Use demo files by clicking 'Build Index', or upload new ones."

        for f in files:
            # In Colab, Gradio often returns NamedString (path-like)
            if isinstance(f, str) or hasattr(f, "__fspath__"):
                file_path = os.fspath(f)
                file_name = os.path.basename(file_path)
                with open(file_path, "rb") as fp:
                    file_bytes = fp.read()

            elif isinstance(f, dict) and "name" in f and "data" in f:
                file_name = f["name"]
                file_bytes = f["data"]

            elif hasattr(f, "name"):
                file_path = getattr(f, "name")
                file_name = os.path.basename(file_path)
                with open(file_path, "rb") as fp:
                    file_bytes = fp.read()

            else:
                return f"❌ Unsupported file object type: {type(f)}"

            uploaded_files_store.append({"name": file_name, "bytes": file_bytes})

        return f"✅ Uploaded {len(uploaded_files_store)} file(s). Now click 'Build Index'."

    except Exception as e:
        return f"❌ Upload failed: {type(e).__name__}: {e}\n{traceback.format_exc()}"


def ui_build_index() -> str:
    global uploaded_files_store

    # fallback to demo docs if no upload
    if not uploaded_files_store and initial_uploaded_files_store:
        uploaded_files_store = initial_uploaded_files_store

    if not uploaded_files_store:
        return "🛑 No files to index. Upload PDFs or ensure demo PDFs exist."

    try:
        return build_index(uploaded_files_store)
    except Exception as e:
        return f"❌ Indexing failed: {type(e).__name__}: {e}\n{traceback.format_exc()}"


## Interactive Audit Interface

The interface brings the full workflow together in a single view.

Auditors can:
- Describe an audit scenario in natural language
- Review a structured audit memorandum
- Inspect the source evidence used
- Verify findings before drawing conclusions

The interface is intentionally simple and transparent, reflecting how audit tools are used in practice.


In [13]:
import gradio as gr
import pandas as pd
import json, traceback
import os, re, uuid
from datetime import datetime

# --- PDF export (memo markdown -> simple PDF) ---
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch

def _memo_md_to_plain_lines(memo_md: str):
    """
    Convert a subset of markdown to plain lines suitable for a PDF.
    Keeps headings and bullets readable.
    """
    lines = []
    for raw in (memo_md or "").splitlines():
        s = raw.strip()
        if not s:
            lines.append("")
            continue

        # Headings
        if s.startswith("### "):
            lines.append(s.replace("### ", "").upper())
            continue
        if s.startswith("## "):
            lines.append(s.replace("## ", "").upper())
            continue
        if s.startswith("# "):
            lines.append(s.replace("# ", "").upper())
            continue

        # Bullets
        if s.startswith("- "):
            lines.append("• " + s[2:])
            continue

        # Remove bold markers
        s = s.replace("**", "")

        lines.append(s)
    return lines


def export_memo_pdf(memo_md: str, filename_prefix: str = "audit_memo") -> str:
    """
    Generates a PDF under a local path and returns the file path for Gradio download.
    """
    export_dir = "exports"
    os.makedirs(export_dir, exist_ok=True)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    uid = uuid.uuid4().hex[:8]
    pdf_path = os.path.join(export_dir, f"{filename_prefix}_{ts}_{uid}.pdf")

    c = canvas.Canvas(pdf_path, pagesize=letter)
    width, height = letter

    x = 0.75 * inch
    y = height - 0.9 * inch

    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(x, y, "Audit Findings Memorandum")
    y -= 0.35 * inch

    c.setFont("Helvetica", 9)
    c.drawString(x, y, f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    y -= 0.35 * inch

    # Body
    c.setFont("Helvetica", 11)
    lines = _memo_md_to_plain_lines(memo_md)

    # Simple word wrap
    def wrap_line(line, max_chars=100):
        if len(line) <= max_chars:
            return [line]
        # basic wrap
        chunks = []
        words = line.split(" ")
        cur = ""
        for w in words:
            if len(cur) + len(w) + 1 <= max_chars:
                cur = (cur + " " + w).strip()
            else:
                chunks.append(cur)
                cur = w
        if cur:
            chunks.append(cur)
        return chunks

    for line in lines:
        if y < 0.8 * inch:
            c.showPage()
            y = height - 0.9 * inch
            c.setFont("Helvetica", 11)

        for wl in wrap_line(line, max_chars=100):
            if y < 0.8 * inch:
                c.showPage()
                y = height - 0.9 * inch
                c.setFont("Helvetica", 11)

            c.drawString(x, y, wl)
            y -= 0.18 * inch

        if line == "":
            y -= 0.05 * inch

    c.save()
    return pdf_path


def evidence_badge(df: pd.DataFrame) -> str:
    if df is None or df.empty:
        return "### Evidence strength: 🔴 Low\nNo source evidence retrieved."

    unique_sources = df["source"].nunique()
    total_chunks = len(df)

    if unique_sources >= 3:
        return (
            "### Evidence strength: 🟢 High\n"
            f"Evidence drawn from {unique_sources} distinct sources "
            f"({total_chunks} relevant excerpts)."
        )

    if unique_sources == 2:
        return (
            "### Evidence strength: 🟡 Medium\n"
            f"Evidence drawn from {unique_sources} sources "
            f"({total_chunks} excerpts)."
        )

    return (
        "### Evidence strength: 🔴 Low\n"
        f"Evidence drawn from a single source ({total_chunks} excerpt(s))."
    )


def ui_ask(query, as_of_date, top_k):
    try:
        result = answer_query(query=query, as_of_date=as_of_date, k=int(top_k))

        sources = result.get("_retrieved_sources", [])
        df = pd.DataFrame(sources) if sources else pd.DataFrame(
            columns=["rank", "source", "chunk_id", "score", "preview"]
        )

        memo = json_to_memo_md(result)
        badge = evidence_badge(df)

        # Return JSON + sources table + memo + badge + state result
        return json.dumps(result, indent=2, ensure_ascii=False), df, memo, badge, result

    except Exception as e:
        err = {
            "error": f"{type(e).__name__}: {str(e)}",
            "traceback": traceback.format_exc()[:6000]
        }
        df = pd.DataFrame(columns=["rank", "source", "chunk_id", "score", "preview"])
        memo = json_to_memo_md(err)
        badge = "### Evidence strength: 🔴 Low\nAn error occurred before evidence could be shown."
        return json.dumps(err, indent=2, ensure_ascii=False), df, memo, badge, err


def show_selected(df: pd.DataFrame, evt: gr.SelectData):
    if df is None or df.empty:
        return "No sources to preview."
    row = df.iloc[evt.index[0]].to_dict()
    return (
        f"Source: {row.get('source')}\n"
        f"Chunk: {row.get('chunk_id')}\n"
        f"Score: {row.get('score')}\n\n"
        f"{row.get('preview')}"
    )


def fill_q1():
    return "Late filing VAT return: what are the penalties and what small business thresholds or exemptions apply?"

def fill_q2():
    return "Provide an audit checklist to verify late filing and late payment, including what evidence to request."

def fill_q3():
    return "What is the corporate income tax rate and what penalties apply?"


def ui_download_pdf(state_result: dict):
    """
    Uses the last result (stored in state) to export the memo as PDF.
    """
    if not isinstance(state_result, dict) or not state_result:
        raise gr.Error("No memo available yet. Please run a query first.")

    memo_md = json_to_memo_md(state_result)
    path = export_memo_pdf(memo_md)
    return path


with gr.Blocks(theme=gr.themes.Soft(), title="Tax Audit Copilot") as demo:
    gr.Markdown("""
# Audit Research & Decision Support Prototype
**Workflow:** Upload PDFs → Build Index → Ask scenario → Review memo + evidence → Download memo as PDF.
""")

    # Holds last result for download
    state_result = gr.State({})

    with gr.Tab("1) Upload & Index"):
        uploader = gr.File(file_count="multiple", label="Upload PDFs/DOCX/TXT")
        upload_status = gr.Textbox(label="Upload status", interactive=False)
        uploader.change(fn=ui_upload, inputs=uploader, outputs=upload_status)

        build_btn = gr.Button("Build Index", variant="primary")
        build_status = gr.Textbox(label="Index status", interactive=False)
        build_btn.click(fn=ui_build_index, inputs=None, outputs=build_status)

    with gr.Tab("2) Ask"):
        with gr.Row():
            with gr.Column(scale=2):
                query = gr.Textbox(
                    label="Audit question / scenario",
                    lines=4,
                    placeholder="Example: Late filing VAT return — penalties, thresholds, exemptions…"
                )

                with gr.Row():
                    demo_q1 = gr.Button("Demo: Late filing + thresholds")
                    demo_q2 = gr.Button("Demo: Audit checklist")
                    demo_q3 = gr.Button("Demo: Refusal test")

                as_of = gr.Textbox(label="As-of date (audit period)", value="Today")
                topk = gr.Slider(3, 10, value=6, step=1, label="Top-K excerpts")

                ask_btn = gr.Button("Generate Audit Summary", variant="primary")

                badge_md = gr.Markdown()

                download_btn = gr.Button("📄 Download Audit Memo (PDF)")
                download_file = gr.File(label="Download", interactive=False)

            with gr.Column(scale=3):
                out_memo = gr.Markdown(label="Audit Findings Memorandum")

        gr.Markdown("---")

        with gr.Row():
            with gr.Column(scale=2):
                out_sources = gr.Dataframe(label="Source Evidence Used (click a row)", interactive=False, wrap=True)
                selected_preview = gr.Textbox(label="Selected Evidence Preview", lines=8, interactive=False)

            with gr.Column(scale=2):
                out_json = gr.Code(label="Structured Output (JSON)", language="json")

        # Events
        demo_q1.click(fill_q1, outputs=query)
        demo_q2.click(fill_q2, outputs=query)
        demo_q3.click(fill_q3, outputs=query)

        ask_btn.click(
            fn=ui_ask,
            inputs=[query, as_of, topk],
            outputs=[out_json, out_sources, out_memo, badge_md, state_result]
        )

        out_sources.select(fn=show_selected, inputs=out_sources, outputs=selected_preview)

        download_btn.click(fn=ui_download_pdf, inputs=state_result, outputs=download_file)


demo.launch(share=False)

/tmp/ipython-input-2815030389.py:204: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="Tax Audit Copilot") as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Final Reflection

This prototype demonstrates how retrieval-based language models can be applied responsibly in audit and regulatory contexts.

Key lessons from this implementation include:
- AI systems are most effective when used as **assistive tools**, not decision-makers
- Transparency and source traceability are essential for trust
- Structured outputs aligned with real audit workflows are more valuable than free-form answers

With appropriate governance, the same approach could be extended to larger document sets, additional tax domains, or integrated audit platforms.

Ultimately, the value of the system lies not in automation alone, but in **supporting better, faster, and more consistent professional judgment**.

**Done by: Abdulla Ahmed Alaydaroos**
